# Importações

In [ ]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from collections import defaultdict
from dateutil.relativedelta import relativedelta
from api_github import listar_arquivos_config

# Criação de Dataframes

In [ ]:
# Arquivos
f_details = 'projects_2025_rpa_by_code_details.csv'
df_details = pd.read_csv(f_details)

In [ ]:
display(df_details)

# Funções

## Filtro por Estrelas

In [ ]:
def filter_stars(df: pd.DataFrame, stars: int) -> pd.DataFrame:
    df_filtered = df[df['stargazers'] >= stars].copy()
    return df_filtered

## Filtro por Atividade

In [ ]:
def filter_pushed(df: pd.DataFrame, m: int) -> pd.DataFrame:
    df['pushedAt'] = pd.to_datetime(df['pushedAt'], utc=True, errors='coerce')
    months_ago = pd.Timestamp.now(tz='UTC') - relativedelta(months=m)
    df_filtered = df[df['pushedAt'] >= months_ago]
    return df_filtered

## Filtro por Issues

In [ ]:
def filter_issues(df: pd.DataFrame, issues: int) -> pd.DataFrame:
    # Garante que a coluna 'issues' está numérica
    df['issues'] = pd.to_numeric(df['issues'], errors='coerce')
    
    # Filtra os repositórios
    df_filtered = df[df['issues'] >= issues]
    
    return df_filtered

## Top Linguagens Mais Usadas

In [ ]:
def plot_linguagens_mais_usadas(df, top_n=10):
    """
    Gera um gráfico de barras com as linguagens mais usadas nos repositórios.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna 'primaryLanguage'
        top_n (int): número de linguagens mais frequentes a exibir (default: 10)
    
    Retorno:
        None (exibe o gráfico)
    """
    # Contar as linguagens mais usadas
    contagem = df['primaryLanguage'].dropna().value_counts().head(top_n)
    
    # Criar o gráfico
    plt.figure(figsize=(10, 6))
    bars = plt.bar(contagem.index, contagem.values, color='mediumseagreen', edgecolor='black')

    # Título e eixos
    plt.title(f"Top {top_n} Linguagens Mais Usadas")
    plt.xlabel("Linguagem")
    plt.ylabel("Número de Repositórios")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Adicionar os rótulos numéricos nas barras
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, height + 0.5, f'{int(height)}', 
                 ha='center', va='bottom', fontsize=9)

    plt.tight_layout()
    plt.show()

## Top Repositórios com Mais Estrelas

In [ ]:
def plot_top_repos_mais_estrelas(df, top_n=10):
    """
    Gera um gráfico de barras horizontal com os repositórios com mais estrelas.
    Exibe a linguagem ao lado do nome do repositório.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame com colunas 'repository', 'primaryLanguage' e 'stargazers'
        top_n (int): número de repositórios a exibir (default: 10)
    
    Retorno:
        None (exibe o gráfico)
    """
    # Garantir que 'stargazers' é numérico
    df['stargazers'] = pd.to_numeric(df['stargazers'], errors='coerce').fillna(0)

    # Criar coluna de rótulo: nome (linguagem)
    df['repositorio'] = df.apply(
        lambda row: f"{row['repository']} ({row['primaryLanguage']})" if pd.notna(row['primaryLanguage']) else row['repository'],
        axis=1
    )

    # Ordenar e pegar os top N
    top_repos = df[['repositorio', 'stargazers']].sort_values(by='stargazers', ascending=False).head(top_n)

    # Plotar gráfico horizontal
    plt.figure(figsize=(10, 6))
    bars = plt.barh(top_repos['repositorio'], top_repos['stargazers'], color='cornflowerblue', edgecolor='black')
    plt.xlabel("Número de Estrelas")
    plt.title(f"Top {top_n} Repositórios com Mais Estrelas")
    plt.gca().invert_yaxis()  # Repositórios mais estrelados no topo
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Adicionar os valores ao lado das barras
    for bar in bars:
        width = bar.get_width()
        plt.text(width + 5, bar.get_y() + bar.get_height()/2, f'{int(width)}', va='center', fontsize=9)

    plt.tight_layout()
    plt.show()


## Extensões mais usadas em arquivos de configuração

In [ ]:
def extensao_mais_frequente_por_repositorio(df):
    """
    Retorna um DataFrame com a extensão mais frequente para cada repositório.
    
    Parâmetros:
    df (pd.DataFrame): DataFrame de entrada contendo as colunas 'repositorio' e 'extensão'
    
    Retorna:
    pd.DataFrame: DataFrame com as colunas 'repositorio' e 'extensão_mais_frequente'
    """
    # Verifica se as colunas necessárias existem no DataFrame
    if not all(col in df.columns for col in ['repositorio', 'extensao']):
        raise ValueError("O DataFrame deve conter as colunas 'repositorio' e 'extensao'")
    
    # Normaliza o nome da extensão .yml
    df.replace(to_replace='.yaml', value='.yml', inplace=True)
    
    # Agrupa por repositório e extensão, conta as ocorrências e pega a extensão mais frequente
    resultado = (
        df.groupby(['repositorio', 'extensao'])
        .size()
        .reset_index(name='contagem')
        .sort_values(['repositorio', 'contagem'], ascending=[True, False])
        .drop_duplicates('repositorio')
        .rename(columns={'extensao': 'extensao_mais_frequente'})
        [['repositorio', 'extensao_mais_frequente']]
    )
    
    return resultado

## Gráfico Rosquinha para o Agrupamento de Extensões

In [ ]:
def plotar_rosquinha_extensoes(df, coluna_extensao, titulo='Distribuição de Extensões', cores=None, sombra=True, explode=0.01):
    """
    Cria um gráfico tipo rosquinha (donut) mostrando a distribuição de extensões de arquivos.
    
    Parâmetros:
    df (pd.DataFrame): DataFrame contendo os dados
    coluna_extensao (str): Nome da coluna com as extensões de arquivo
    titulo (str): Título do gráfico (opcional)
    cores (list): Lista de cores para as fatias (opcional)
    sombra (bool): Se True, adiciona sombra ao gráfico (opcional)
    explode (float): Valor para separar as fatias (0 para nenhuma separação)
    """
    
    # Verifica se a coluna existe no DataFrame
    if coluna_extensao not in df.columns:
        raise ValueError(f"A coluna '{coluna_extensao}' deve existir no DataFrame")
    
    # Calcula a contagem de cada extensão
    contagem_extensoes = df[coluna_extensao].value_counts().reset_index()
    contagem_extensoes.columns = ['Extensao', 'Quantidade']
    
    # Se houver muitas extensões, agrupa as menos frequentes em "Outros"
    if len(contagem_extensoes) > 10:
        top_extensoes = contagem_extensoes.head(9)
        outras = pd.DataFrame({
            'Extensao': ['Outros'],
            'Quantidade': [contagem_extensoes['Quantidade'][9:].sum()]
        })
        contagem_extensoes = pd.concat([top_extensoes, outras])
    
    # Configuração do gráfico
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Cria o gráfico de rosquinha
    wedges, texts, autotexts = ax.pie(
        contagem_extensoes['Quantidade'],
        labels=contagem_extensoes['Extensao'],
        autopct=lambda p: '{:.1f}%\n({:.0f})'.format(p, p * sum(contagem_extensoes['Quantidade']) / 100),
        startangle=90,
        colors=cores,
        shadow=sombra,
        explode=[explode] * len(contagem_extensoes),
        wedgeprops={'width': 0.4, 'edgecolor': 'white'},
        textprops={'fontsize': 9}
    )
    
    # Ajusta o estilo dos textos - Versão com alinhamento modificado
    plt.setp(autotexts, size=8, weight="bold", color='black',
        horizontalalignment='left', verticalalignment='center')
    plt.setp(texts, size=9, horizontalalignment='center', verticalalignment='center')
    
    # Adiciona título e legenda
    ax.set_title(titulo, pad=20, fontsize=14, fontweight='bold')
    plt.legend(
        wedges,
        contagem_extensoes['Extensao'],
        title="Extensões",
        loc="center left",
        bbox_to_anchor=(1, 0, 0.5, 1))
    
    # Garante que o gráfico seja desenhado como círculo
    ax.axis('equal')
    
    # Mostra o gráfico
    plt.tight_layout()
    plt.show()

## Gráfico de Pizza para a Proporção do Uso de Diretório de Configuração

In [ ]:
def plot_pizza_coluna_booleana(df, coluna, titulo="Distribuição booleana"):
    """
    Gera um gráfico de pizza para uma coluna booleana de um DataFrame.
    
    Parâmetros:
        df (pd.DataFrame): DataFrame contendo a coluna booleana
        coluna (str): nome da coluna booleana
        titulo (str): título do gráfico (opcional)
    
    Retorno:
        None (exibe o gráfico)
    """
    # Contar valores True e False
    contagem = df[coluna].value_counts().sort_index()

    # Criar rótulos amigáveis
    labels = ['Não utilizam', 'Utilizam']
    valores = [contagem.get(False, 0), contagem.get(True, 0)]

    # Plotar
    plt.figure(figsize=(6, 6))
    plt.pie(valores, labels=labels, autopct='%1.1f%%', colors=['lightcoral', 'mediumseagreen'], startangle=90)
    plt.title(titulo)
    plt.axis('equal')  # Mantém o círculo redondo
    plt.show()

# Análises

## Parâmetros

In [ ]:
stars = 250
months = 15
token = 'my_token'

## Filtros

In [ ]:
print('Universo total de amostras:')
df_filter = df_details.copy()
display(df_filter)

print(f'Repositórios com {stars} ou mais estrelas:')
df_filter = filter_stars(df_filter, stars)
display(df_filter)

print(f'Repositórios com atualizações nos últimos {months} meses:')
df_filter = filter_pushed(df_filter, months)
display(df_filter)

In [ ]:
display(df_filter[df_filter['issues'] == 0])

## Gráficos

In [ ]:
plot_linguagens_mais_usadas(df_filter, top_n=10)

In [ ]:
plot_top_repos_mais_estrelas(df_filter, top_n=10)

In [ ]:
# Carregar a planilha (ajuste o nome do arquivo se necessário)
df = df_details_stars.copy()

# Converter as datas para datetime (considerando formato ISO 8601)
df['createdAt'] = pd.to_datetime(df['createdAt'], utc=True, errors='coerce')
df['pushedAt'] = pd.to_datetime(df['pushedAt'], utc=True, errors='coerce')

# Obter a data atual em UTC
now = pd.Timestamp.now(tz='UTC')

# Calcular idade em meses (aproximação: 1 mês = 30,44 dias)
df['repo_age_months'] = ((now - df['createdAt']).dt.total_seconds()) / (60 * 60 * 24 * 30.44)

# Calcular tempo desde o último push (em dias)
df['days_since_last_push'] = (now - df['pushedAt']).dt.days

# Exibir estatísticas
print("Resumo estatístico da idade dos repositórios (meses):")
print(df['repo_age_months'].describe())

print("\nResumo estatístico do tempo desde o último push (dias):")
print(df['days_since_last_push'].describe())

# Gráfico da idade dos repositórios
plt.figure(figsize=(10, 5))
plt.hist(df['repo_age_months'].dropna(), bins=20, color='steelblue', edgecolor='black')
plt.title("Distribuição da Idade dos Repositórios (em meses)")
plt.xlabel("Idade (meses)")
plt.ylabel("Quantidade de Repositórios")
plt.grid(True)
plt.show()

# Gráfico do tempo desde o último push
plt.figure(figsize=(10, 5))
plt.hist(df['days_since_last_push'].dropna(), bins=20, color='darkorange', edgecolor='black')
plt.title("Tempo Desde o Último Push (em dias)")
plt.xlabel("Dias desde o último push")
plt.ylabel("Quantidade de Repositórios")
plt.grid(True)
plt.show()


In [ ]:
display(df)

## Perguntas

### Quais são os arquivos de configuração mais utilizados?

#### Criar o dataframe com informações sobre arquivos de configuração

In [ ]:
# Inicializar listas para coletar dados
dados = []

# Define pastas consideradas "organizadoras de configuração"
pastas_config = ('config/', 'configs/', 'settings/', 'env/', 'environments/')

# Loop pelos repositórios
for idx, repo in df_filter['repository'].items():
    # print(f'\n--- {repo} ---\n')

    arquivos_config = listar_arquivos_config(repo, token)

    if arquivos_config:
        # print(f"Arquivos de configuração encontrados em '{repo}':")
        for arq in arquivos_config:
            # print(" -", arq)

            # Extrai extensão (ou '[sem_extensao]' se não tiver)
            extensao = os.path.splitext(arq)[1].lower() or '[sem_extensao]'

            # Verifica se está em uma pasta de configuração
            em_pasta_config = any(arq.lower().startswith(pasta) for pasta in pastas_config)

            # Armazena no formato estruturado
            dados.append({
                'repositorio': repo,
                'arquivo': arq,
                'extensao': extensao,
                'em_pasta_config': em_pasta_config
            })
    else:
        print(f"Nenhum arquivo de configuração encontrado em '{repo}'.")

# Criar DataFrame com os resultados
df_config = pd.DataFrame(dados)

# Exibir um resumo dos dados
# print("\nAmostra dos arquivos de configuração identificados:")
# print(df_config.head())

# Contagem por tipo de extensão
# print("\nExtensões mais comuns:")
# print(df_config['extensao'].value_counts())

# Porcentagem de arquivos organizados em pastas de configuração
# percentual_organizados = df_config['em_pasta_config'].mean() * 100
# print(f"\n{percentual_organizados:.1f}% dos arquivos de configuração estão organizados em pastas dedicadas.")


In [ ]:
display(df_config)

#### Agrupa o dataframe de acordo o tipo de arquivo

In [ ]:
df_group = extensao_mais_frequente_por_repositorio(df_config)
display(df_group)

#### Gera o gráfico das extensões agrupadas

In [ ]:
plotar_rosquinha_extensoes(df=df_group, coluna_extensao='extensao_mais_frequente', titulo='Principais Extensões em Arquivos de Configuração por Repositório')

#### Agrupa o dataframe de acordo com uso de um diretório dedicado

In [ ]:
df_group = df_config[['repositorio','em_pasta_config']].copy()
df_group = pd.DataFrame(df_group.groupby('repositorio')['em_pasta_config'].any())
display(df_group)

#### Gera o gráfica da proporção gerada

In [ ]:
plot_pizza_coluna_booleana(df=df_group,coluna='em_pasta_config',titulo='Proporção de Repositórios que Utilizam um Diretório Dedicado a Arquivos de Configuração')